In [ ]:
import cv2
import numpy as np
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

# ---------------------------------------------------------
# 1. 캘리브레이션 데이터 로드 및 맵 생성 (최초 1회만 실행)
# ---------------------------------------------------------
calib_data = np.load('fisheye_calib.npz')
K = calib_data['K']
D = calib_data['D']

# 카메라 해상도 설정 (데이터 수집 때와 동일하게)
WIDTH, HEIGHT = 224, 224

# 맵 계산 (이 부분이 무거우므로 루프 밖에서 미리 계산합니다)
# balance: 0.0 (검은색 제거/확대), 1.0 (전체 보존) -> 원하는 대로 조절
balance = 0.0 
new_K = cv2.fisheye.estimateNewCameraMatrixForUndistortRectify(
    K, D, (WIDTH, HEIGHT), np.eye(3), balance=balance
)
map1, map2 = cv2.fisheye.initUndistortRectifyMap(
    K, D, np.eye(3), new_K, (WIDTH, HEIGHT), cv2.CV_16SC2
)

# ---------------------------------------------------------
# 2. 카메라 및 디스플레이 위젯 설정
# ---------------------------------------------------------
camera = Camera.instance(width=int(3280/4), height=int(2464/4))

# 원본과 보정본을 보여줄 위젯 생성
widget_original = widgets.Image(format='jpeg', width=WIDTH, height=HEIGHT)
widget_undistorted = widgets.Image(format='jpeg', width=WIDTH, height=HEIGHT)

# 나란히 배치 (HBox)
display(widgets.HBox([widgets.Label("원본"), widget_original]))
display(widgets.HBox([widgets.Label("보정본"), widget_undistorted]))


# ---------------------------------------------------------
# 3. 실시간 처리 함수 (콜백)
# ---------------------------------------------------------
def update_frame(change):
    frame = change['new']  # 카메라의 새로운 프레임 (numpy array)
    
    # [핵심] 미리 계산된 map을 이용해 빠르게 변환
    dst = cv2.remap(frame, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
    
    # 화면 업데이트 (JPEG로 변환하여 위젯에 표시)
    widget_original.value = bgr8_to_jpeg(frame)
    widget_undistorted.value = bgr8_to_jpeg(dst)

# ---------------------------------------------------------
# 4. 카메라와 처리 함수 연결 (Start)
# ---------------------------------------------------------
# 기존에 연결된 게 있다면 끊어줍니다 (에러 방지)
camera.unobserve_all()

# 카메라 값이 변할 때마다 update_frame 함수를 실행
camera.observe(update_frame, names='value')